In [17]:
from pathlib import Path
import os
import sys
if '..' not in sys.path:
    sys.path.append('..')

import numpy as np
import open3d as o3d

from sdp.ds.bop_data import BopModelsInfo, read_models_info

In [18]:
DATA_PATH = Path(os.path.expandvars('$HOME/data'))
BOP_PATH = DATA_PATH / 'bop'
ITODD_SUBDIR = 'itodd'
ITODD_BOP_PATH = BOP_PATH / ITODD_SUBDIR
print(f'BOP path: {BOP_PATH}')

BOP path: /Users/misha/data/bop


In [19]:
obj_path = DATA_PATH / 'sds_data' / 'objs' / 'teamug.stl'
mesh = o3d.io.read_triangle_mesh(str(obj_path), True)

In [20]:
mesh.vertices, mesh.triangles

(std::vector<Eigen::Vector3d> with 129251 elements.
 Use numpy.asarray() to access data.,
 std::vector<Eigen::Vector3i> with 59328 elements.
 Use numpy.asarray() to access data.)

In [21]:
vs, ts = np.asarray(mesh.vertices), np.asarray(mesh.triangles)
print(vs.shape, vs.dtype, vs.min(), vs.mean(), vs.max())
print(ts.shape, ts.dtype, ts.min(), ts.mean(), ts.max())

(129251, 3) float64 -0.24444793164730072 -0.00841415762465798 0.2220826894044876
(59328, 3) int32 0 63898.17107155699 129250


In [23]:
models_path = ITODD_BOP_PATH / 'models'
models_info_path = models_path / 'models_info.json'
models_info = read_models_info(models_info_path)
